# 시계열 데이터 이상탐지란

이상치란 **'다른 방법에 의해 생성되었다는 의심이 될 정도로 관찰에서 너무 많이 벗어난 관찰이다.'** 따라서 이상값이란 예상되는 동작에 따르지 않는 관측치로 볼 수 있다.

![Outlier](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Time-series-outliers.png?ssl=1)

시계열에서 이상값은 두 가지 다른 의미를 갖는다. 원치않은 데이터일 경우와 특정 사건에 의한 데이터이다.

</br>

시계열 데이터에서 이러한 이상값 탐지에 대표적인 예시는 *사기 탐지*다. 주요 목표가 이상치 자체를 탐지하고 분석하는 것이다. 시계열에서 이상감지는 'Point Outlier', 'Subsequence Outlier' 두 유형으로 분류 할 수 있다.

![Outlie2](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Time-series-outliers-types.png?resize=587%2C377&ssl=1)


# Point Outlier
Point Outlier는 시계열의 다른 값 또는 인접한 값들과 비교할 때 특정 시간에서 비정상적인 데이터를 말한다.

![Point Outlier](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Point-outliers-in-time-series.png?resize=750%2C301&ssl=1)

# Subsequence Outlier
Subsequence Outlier는 같은 움직임이 비정상적인 시간의 연속적인 현상을 의미한다. 전역 또는 로컬일 수 있으며 하나 또는 그 이상 시간 종속 변수에 영향을 줄 수도 있다.

![Ss Outlier](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/%E2%80%8ASubsequence-outliers-in-time-series.png?ssl=1)

</br></br>

## 시계열 데이터의 이상 감지 기술

### STL 분해
STL(Seasonal and Trend decomposition using Loess) Loess는 비선형 관계를 추정하기 위한 기법이다. 

![STL](https://otexts.com/fppkr/fpp_files/figure-html/elecequip-stl2-1.png)


STL에서는 t.window(Trend), s.window(Seasonal)의 매개변수에 따라 추세-주기와 계절성을 조정할 수 있다. 


### 분류 및 회귀 트리(CART)

의사결정 나무의 훌륭한 분류 기능을 통해 시계열 데이터에서의 이상값을 식별할 수 있다. 그 방법은

- 의사결정 나무 모델에 이상과 비이상을 분류할 수 있는 포인트를 학습 시킬 수 있다. 이를 위해서 이상치 데이터에 대한 레이블링이 필요하고 이 데이터는 토이 데이터셋 외부에서는 쉽게 발견할 수 없어야 한다.

- 학습시키지 않은 데이터도 필요하다. Isolation Forest 알고리즘을 사용하여 레이블이 지정된 데이터 집합의 도움 없이 특정 포인트가 이상치인지 아닌지 예측할 수 있다.

핵심은 Isolation Forest는 다른 이상값 검색 방법과 달리 일반 데이터를 프로파일링 하는 대신에 명시적으로 이상치를 식별한다. Isolation Forest는 다른 앙상블 기법과 마찬가지로 의사결정 트리를 기반으로 한다. 즉, Isolation Forest는 약간의 다른 데이터 포인트라는 사실을 기반으로 이상치를 감지한다. 또한, 거리 또는 밀도 측정을 사용하지 않고 동작한다.

- Isolation Forest 모델을 만들때 (contamination = outliers_fraction)옵션을 세팅하는데 이는 현재 데이터에 이상값의 비율을 모델에 알려주는 것입니다. 이는 trial/error 메트릭스이다.
- 데이터에 대한 학습과 예측을 진행하면 정상에 대해서는 1을 이상치에 대해서는 -1을 반환한다.
- 마지막으로 이상치를 시계열로 시각화한다.

~~~
plt.rc('figure',figsize=(12,6))
plt.rc('font',size=15)
catfish_sales.plot()
~~~


![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Time-series-anomalies.png?ssl=1)


~~~
#전처리
outliers_fraction = float(.01)
scaler = StandardScaler()
np_scaled = scaler.fit_transform(catfish_sales.values.reshape(-1, 1))
data = pd.DataFrame(np_scaled)

# 모델 학습
model =  IsolationForest(contamination=outliers_fraction)
model.fit(data)


catfish_sales['anomaly'] = model.predict(data)

# 시각화
fig, ax = plt.subplots(figsize=(10,6))
a = catfish_sales.loc[catfish_sales['anomaly'] == -1, ['Total']] #이상치
ax.plot(catfish_sales.index, catfish_sales['Total'], color='black', label = 'Normal')
ax.scatter(a.index,a['Total'], color='red', label = 'Anomaly')
plt.legend()
plt.show();
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Anomaly-Detection-Isolation-Forest.png?ssl=1)

그림을 보면 굉장히 아상치에 탐지를 잘 했음을 알 수 있다. 하지만 시자 지점에서의 이상치 탐지가 좋지 못한 결과같아 보인다. 이에 대한 이유는 두 가지가 있다.
</br>
- 처음에는 알고리즘이 이상치로 판단할 수 있는 데이터가 많지 않기 떄문인데 이는 많은 데이터를 얻을수록 더 많은 분산을 볼 수 있게 되고 스스로 그 범위를 조정하기 때문
- 이런 값들이 많이 보인다면 모델의 contamination 파라미터를 너무 높게 설정했기 때문이다.
</br>

**장점**</br>
이 기술의 가장 큰 이점은 더욱 정교한 모델을 만들기 위해서 원하는 만큼 많은 변수나 피처를 넣을 수 있다는 것입니다.


**단점**</br>
단점은 피처가 증가함에 따라서 컴퓨터 퍼포먼스에 영향을 준다. 이럴 경우 피처 선택을 신중하게 해야합니다.

</br></br>

### 예측을 사용한 이상치 탐지
미래 예측을 통한 이상치 감지는 과거의 데이터에 white noise를 추가하여 미래에 대한 예측을 생성하는 접근 방식이다. 미래에 대한 예측에 대한 선은 부드러운 형태일 것입니다.</br>
이 방법을 사용할때 어려운 점은 차이의 수, 자기회귀 계수, 예측 오류 계수를 선택해야한다는 것입니다.
</br>
이는 새로운 신호의 작업을 할때마다 새로운 예측 모델을 만들어야합니다.
</br>
또 다른 장애요소는 차이를 확인한 후 신호가 고정되어 있어야 한다는 것입니다. 간단히 말해, 신호가 시간에 의존해서는 안됩니다. 이는 중요한 제약 조건입니다.
</br>
이동평균(Moving Average), 자기회귀(Autoregress) 접근 방식 및 ARIMA와 같은 방법으로 활용할 수 있습니다. ARIMA를 통한 이상 탐지 방법은 아래와 같다.
- 과거 데이터를 통해 예측과 훈련 데이터와의 크기 차이가 있다.
- 임계값(theshold)를 설정하고 임계값과 다름을 기준으로 이상치를 식별한다.
</br>

이를 위해 Prophet 모듈을 사용합니다.

~~~
def fit_predict_model(dataframe, interval_width = 0.99, changepoint_range = 0.8):
   m = Prophet(daily_seasonality = False, yearly_seasonality = False, weekly_seasonality = False,
               seasonality_mode = 'additive',
               interval_width = interval_width,
               changepoint_range = changepoint_range)
   m = m.fit(dataframe)
   forecast = m.predict(dataframe)
   forecast['fact'] = dataframe['y'].reset_index(drop = True)
   return forecast

pred = fit_predict_model(t)
~~~

아래 코드는 Prophet에서 제공하는 interval_width를 가져오는 과정입니다.

~~~
def detect_anomalies(forecast):
   forecasted = forecast[['ds','trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact']].copy()
forecasted['anomaly'] = 0
   forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
   forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = -1
#anomaly importances
   forecasted['importance'] = 0
   forecasted.loc[forecasted['anomaly'] ==1, 'importance'] =
       (forecasted['fact'] - forecasted['yhat_upper'])/forecast['fact']
   forecasted.loc[forecasted['anomaly'] ==-1, 'importance'] =
       (forecasted['yhat_lower'] - forecasted['fact'])/forecast['fact']

   return forecasted
pred = detect_anomalies(pred)
~~~

이를 시각화하면 아래와 같습니다.

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Anomaly-detection.png?ssl=1)

**장점**</br>
이 방법은 월간, 연간과 같은 다양한 계절성 매개변수에 대해서 훌륭한 결과를 보여줍니다. Isolation Forest와 비교한다면 엣지 케이스에 대한 처리가 좋습니다.

</br>
**단점**</br>
이 방법은 예측을 기반으로 하기 때문에 제한된 데이터를 갖고 있는 경우 어려움이 있습니다. 제한된 데이터는 곧 예측 품질이 낮아지고 이상 감지 정확도가 낮이지기 떄문입니다.

### 클러스터링 기반 이상 탐지


